## Code for automatic merging of Mozambique data

Some code to process XLS data and merge it into an appropriate format.

**After longer work on this - retreat! - do manual merging**

In [147]:
import os
import xlrd
import numpy as np
import pandas as pd
from termcolor import colored
import unidecode

In [52]:
file_path = "C://Users//wb555300//OneDrive - WBG//Municipality_merge//Source_tables"
os.chdir(file_path)
os.getcwd()

'C:\\Users\\wb555300\\OneDrive - WBG\\Municipality_merge\\Source_tables'

#### Functions:

In [53]:
#Create a function to check if format is xls:
def is_xls(files):
    xls_format = []
    for x in files:
        leng = len(x.strip().split("."))
        xls_format.append(x.strip().split(".")[leng-1]=="xls")
    return np.array(xls_format)


# split at multiple different seperators:
def multi_split(txt, seps):
    default_sep = seps[0]

    # we skip seps[0] because that's the default separator
    for sep in seps[1:]:
        txt = txt.replace(sep, default_sep)
    return [i.strip() for i in txt.split(default_sep)]


## Functions to identify non-empty rows and columns:

def nonempty_rows(sheet):
    empty_row = []
    for row in range(sheet.nrows):
        empty_row.append(all(np.array(sheet.row_values(row,0))==''))
    return np.where(np.array(empty_row)==False)[0]

def nonempty_cols(sheet):
    empty_col = []
    for col in range(sheet.ncols):
        empty_col.append(all(np.array(sheet.col_values(col,0))==''))
    return np.where(np.array(empty_col)==False)[0]



#### Code:

In [100]:
#Make list with all files in directory
all_files = np.array(os.listdir())
xls_files = all_files[is_xls(all_files)]
print(xls_files)

['BEIRA_ Informação Financeira Municipal 2013 a 2019.xls'
 'CHIBUTO  Informação Financeira Municipal 2013 a 2019.xls'
 'CHIMOIO_ Informação Financeira Municipal 2013 a 2019.xls'
 'DONDO_ Financeira Municipal 2013 a 2019.xls'
 'GONDOLA Informação Financeira Municipal 2013 a 2019.xls'
 'GORONGOSA_Informação Financeira Municipal 2013 a 2019.xls'
 'GURUE_RECEITA 2013-2019.xls'
 'LICHINGA_Colecta de  Informação Financeira Municipal 2013 a JUNHO 2019.xls'
 'MALEMA_EXECUCAO FINANCEIRA DA AUTARQUIA 2013-2019.xls'
 'MANDIMBA_Info.xls'
 'MANDLAKAZI_ Informação Financeira Municipal 2013 a 2019.xls'
 'MAPUTO Evolução da Execução Orçamental 2013 - 2019.xls'
 'MARROMEU_ Informação Financeira Municipal 2013 a 2019.xls'
 'MASSINGA_  InformaÃ§Ã£o Financeira Municipal 2013 a JUNHO 2019.xls'
 'MATOLA_ Informação Financeira Municipal 2013 a 2019.xls'
 'MAXIXE  Informação Financeira Municipal 2013 a 2019.xls'
 'MOCIMBOA DA PRAIA_COLECTA DE INFOR. 2013-2019.xls'
 'MOCUBA_Informação Financeira Municipal 2013

In [55]:
xls_files[0:2]

array(['BEIRA_ Informação Financeira Municipal 2013 a 2019.xls',
       'CHIBUTO  Informação Financeira Municipal 2013 a 2019.xls'],
      dtype='<U74')

### Extract all tables from one file:

Extract current municipality name:

In [200]:
## Save list with problematic files
problem_files = []

## List with municipalities:
munis = []
# and available municipalities
munis_avail = []


for file in xls_files:

    #Extract muni name:
    muni_name = multi_split(file, seps = [" ", "_", "."])[0]
    munis.append(muni_name)
    print('Currently processing data on: ', muni_name)
    
    #Open file:
    wb = xlrd.open_workbook(file)
    sheet = wb.sheet_by_index(0) 

    # Extract number of rows and columns in file:
    ne_rows = nonempty_rows(sheet)
    ne_cols = nonempty_cols(sheet)

    # identify rows and columns where table starts (all tables tart with DESCRICAO):
    start_rowcol = []

    for row in ne_rows:
        for col in ne_cols:
            if(sheet.cell_value(row, col)=="DESCRICAO"): 
                start_rowcol.append([row, col])


    #Extract all tables:
    table_count = 1

    for c_start in start_rowcol:

        year_row = c_start[0]
        OE_row = c_start[0]+1

        years = sheet.row_values(year_row, c_start[1]+1)
        OEs = sheet.row_values(OE_row, c_start[1]+1)
        OEs = list(map(lambda x: unidecode.unidecode(x), OEs))

        # dupicate year vector:
        for x in range(len(years)):
            if years[x] == '':
                years[x] = years[x-1]

        i = 2
        current_rowname_start = multi_split(sheet.cell_value(c_start[0]+i, c_start[1]), [" ", "_", "."])[0]

        temp_df = pd.DataFrame({'muni': np.repeat(muni_name, len(years)),'year': np.array(years), 'OE': np.array(OEs)})
        temp_df_cols = 3

        while current_rowname_start != "TOTAL":
            current_rowname_start = multi_split(sheet.cell_value(c_start[0]+i, c_start[1]), [" ", "_", "."])[0]
            temp_name = unidecode.unidecode(sheet.cell_value(c_start[0]+i, c_start[1])).upper()
            temp_values = sheet.row_values(c_start[0]+i, c_start[1]+1)
                      
            temp_df.insert(temp_df_cols, temp_name, temp_values, True)

            temp_df_cols = temp_df_cols +1
            i = i+1

        csv_file_path = 'C:/Users/wb555300/OneDrive - WBG/Municipality_merge/' + muni_name + '_Table_' + str(table_count) +'.csv'
    
        temp_df.to_csv(csv_file_path)
        table_count = table_count + 1
     
    if table_count-1 != 3: 
        print(colored(str(table_count-1) + ' tables found \n', color='red'))
        problem_files.append(file)
    else:     
        print(colored(str(table_count-1) + ' tables found \n'))
        munis_avail.append(muni_name)
        
    

Currently processing data on:  BEIRA
3 tables found 

Currently processing data on:  CHIBUTO
3 tables found 

Currently processing data on:  CHIMOIO
3 tables found 

Currently processing data on:  DONDO
3 tables found 

Currently processing data on:  GONDOLA
3 tables found 

Currently processing data on:  GORONGOSA
3 tables found 

Currently processing data on:  GURUE
0 tables found 

Currently processing data on:  LICHINGA
3 tables found 

Currently processing data on:  MALEMA
3 tables found 

Currently processing data on:  MANDIMBA
4 tables found 

Currently processing data on:  MANDLAKAZI
3 tables found 

Currently processing data on:  MAPUTO
0 tables found 

Currently processing data on:  MARROMEU
3 tables found 

Currently processing data on:  MASSINGA
3 tables found 

Currently processing data on:  MATOLA
3 tables found 

Currently processing data on:  MAXIXE
3 tables found 

Currently processing data on:  MOCIMBOA
3 tables found 

Currently processing data on:  MOCUBA
3 tables f

['ORCAMENTO',
 'EXECUCAO',
 'ORCAMENTO',
 'EXECUCAO',
 'ORCAMENTO',
 'EXECUCAO',
 'ORCAMENTO',
 'EXECUCAO',
 'ORCAMENTO',
 'EXECUCAO',
 'ORCAMENTO',
 'EXECUCAO',
 'ORCAMENTO',
 'EXECUCAO (ATE JUNHO)']

In [120]:
print(problem_files)
print(munis)
print(munis_avail)


['GURUE_RECEITA 2013-2019.xls', 'MANDIMBA_Info.xls', 'MAPUTO Evolução da Execução Orçamental 2013 - 2019.xls', 'MOLOCUE Informação Financeira 2013-2019.xls']
['BEIRA', 'CHIBUTO', 'CHIMOIO', 'DONDO', 'GONDOLA', 'GORONGOSA', 'GURUE', 'LICHINGA', 'MALEMA', 'MANDIMBA', 'MANDLAKAZI', 'MAPUTO', 'MARROMEU', 'MASSINGA', 'MATOLA', 'MAXIXE', 'MOCIMBOA', 'MOCUBA', 'MOLOCUE', 'MONAPO', 'MUEDA', 'NACALA', 'NHAMATANDA', 'QUELIMANE', 'QUISSICO1', 'TETE', 'VILANKULO']
['BEIRA', 'CHIBUTO', 'CHIMOIO', 'DONDO', 'GONDOLA', 'GORONGOSA', 'LICHINGA', 'MALEMA', 'MANDLAKAZI', 'MARROMEU', 'MASSINGA', 'MATOLA', 'MAXIXE', 'MOCIMBOA', 'MOCUBA', 'MONAPO', 'MUEDA', 'NACALA', 'NHAMATANDA', 'QUELIMANE', 'QUISSICO1', 'TETE', 'VILANKULO']


In [180]:
unidecode.unidecode(open_table("VILANKULO", 1).OE[1])


open_table("VILANKULO", 1).iloc[:,4].apply(lambda x: unidecode.unidecode(str(x)))



open_table("VILANKULO", 1).rename(unidecode.unidecode, axis = 'columns')


,muni,year,OE,RECEITAS PROPRIAS,RECEITAS FISCAIS,RECEITAS NAO FISCAIS,RENDIMENTO DE BENS MOVEIS E IMOVEIS,"RECEITAS EXTERNAS ( 2,1+2,2)",TRANSFERENCIAS DO ESTADO,FCA,...,PROMAPUTO,OUTROS,PCMC,COOP. ESPANHOLA,ORCAMENTACAO PARTICIPATIVA,P13,PDA,PRODEM,UNIAO EUROPEIA,TOTAL DE RECEITAS PROPRIAS (1+2)
0,VILANKULO,2013.0,ORÇAMENTO,10306720.00,4581720.00,5725000.00,0.0,36703360.00,28258000.00,14992690.00,...,0.0,600000.0,2200000.00,0.0,0.0,0.0,0.0,0.0,5645360.00,47010080.00
1,VILANKULO,2013.0,EXECUÇÃO,8689710.00,2493430.00,6196280.00,0.0,35821490.00,28258000.00,14992690.00,...,0.0,600000.0,2000000.00,0.0,0.0,0.0,0.0,0.0,4963490.00,44511200.00
2,VILANKULO,2014.0,ORÇAMENTO,10615000.00,2550000.00,8065000.00,0.0,38136374.03,32937634.03,18488665.24,...,0.0,0.0,2872610.00,0.0,0.0,0.0,0.0,0.0,2326130.00,48751374.03
3,VILANKULO,2014.0,EXECUÇÃO,7457850.28,1761992.44,5695857.84,0.0,32736373.69,28337634.03,18488665.24,...,0.0,0.0,2072610.00,0.0,0.0,0.0,0.0,0.0,2326129.66,40194223.97
4,VILANKULO,2015.0,ORÇAMENTO,11062000.00,2989000.00,8073000.00,0.0,51549980.00,39122050.00,18593200.00,...,0.0,4000000.0,6201110.00,0.0,0.0,0.0,0.0,0.0,2226820.00,62611980.00
5,VILANKULO,2015.0,EXECUÇÃO,9735026.57,2387441.57,7347585.00,0.0,34103360.00,30497860.00,18593200.00,...,0.0,40000.0,3564900.00,0.0,0.0,0.0,0.0,0.0,600.00,43838386.57
6,VILANKULO,2016.0,ORÇAMENTO,10962000.00,3029000.00,7933000.00,0.0,59576620.00,47337410.00,20192540.00,...,0.0,4000000.0,8239210.00,0.0,0.0,0.0,0.0,0.0,0.00,70538620.00
7,VILANKULO,2016.0,EXECUÇÃO,11366610.00,3740770.00,7625840.00,0.0,45077030.00,37949170.00,20192540.00,...,0.0,4000000.0,3127860.00,0.0,0.0,0.0,0.0,0.0,0.00,56443640.00
8,VILANKULO,2017.0,ORÇAMENTO,12146920.00,4108920.00,8038000.00,0.0,64928610.00,58230020.00,21745090.00,...,0.0,0.0,6698590.00,0.0,0.0,0.0,0.0,0.0,0.00,77075530.00
9,VILANKULO,2017.0,EXECUÇÃO,10739251.89,3593404.40,7145847.49,0.0,42621699.00,41049970.00,21745090.00,...,0.0,0.0,1571729.00,0.0,0.0,0.0,0.0,0.0,0.00,53360950.89


In [203]:
# Code to open a respective table:

def open_table(muni, table, path = 'C:/Users/wb555300/OneDrive - WBG/Municipality_merge/'):
    path_loc = path + muni + '_Table_' + str(table) +'.csv'
    return pd.read_csv(path_loc, index_col=0)


## check which columns are available in all data sets:

overview_table_1 = pd.DataFrame()

for muni in munis_avail:
    temp_table = open_table(muni,1)
    overview_table_1 = overview_table_1.append(temp_table, sort = False)

overview_table_1.to_csv("../Overview1.csv")




In [145]:
overview_table_1 = pd.DataFrame()
overview_table_1 = overview_table_1.append(open_table("BEIRA",1))
overview_table_1

,muni,year,OE,RECEITAS PROPRIAS,RECEITAS FISCAIS,RECEITAS NAO FISCAIS,RENDIMENTO DE BENS MOVEIS E IMOVEIS,"RECEITAS EXTERNAS ( 2,1+2,2)",TRANSFERENCIAS DO ESTADO,FCA,...,PROMAPUTO,OUTROS (DONATIVOS),PCMC,COOP. ESPANHOLA,ORCAMENTACAO PARTICIPATIVA,P13,PDA,PRODEM,UNIAO EUROPEIA,TOTAL DE RECEITAS PROPRIAS (1+2)
0,BEIRA,2013.0,ORÇAMENTO,3.680504e+08,1.037888e+08,2.626616e+08,1600000.0,3.880842e+08,2.523196e+08,1.477978e+08,...,0.0,1.357646e+08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.561346e+08
1,BEIRA,2013.0,EXECUÇÃO,2.496427e+08,6.587861e+07,1.824772e+08,1286900.0,2.734374e+08,2.523197e+08,1.477980e+07,...,0.0,2.111774e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.230801e+08
2,BEIRA,2014.0,ORÇAMENTO,4.139027e+08,1.153200e+08,2.968707e+08,1712000.0,3.506254e+08,2.888420e+08,1.744014e+08,...,0.0,6.178341e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.645282e+08
3,BEIRA,2014.0,EXECUÇÃO,2.636741e+08,8.927405e+07,1.729861e+08,1413900.0,3.177486e+08,2.747058e+08,1.822610e+08,...,0.0,4.304287e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.814227e+08
4,BEIRA,2015.0,ORÇAMENTO,4.507108e+08,1.334190e+08,3.154086e+08,1883200.0,3.389403e+08,3.167569e+08,1.832915e+08,...,0.0,2.218341e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.896511e+08
5,BEIRA,2015.0,EXECUÇÃO,3.187132e+08,1.038807e+08,2.130947e+08,1737700.0,3.003485e+08,3.003485e+08,1.832915e+08,...,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.190616e+08
6,BEIRA,2016.0,ORÇAMENTO,4.234907e+08,1.226465e+08,2.986442e+08,2200000.0,3.965245e+08,3.436483e+08,1.990578e+08,...,0.0,5.287616e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.200152e+08
7,BEIRA,2016.0,EXECUÇÃO,3.206264e+08,6.704294e+07,2.513185e+08,2265000.0,3.351027e+08,3.202398e+08,1.990787e+08,...,0.0,1.486282e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.557291e+08
8,BEIRA,2017.0,ORÇAMENTO,4.899419e+08,4.438343e+08,4.335761e+07,2750000.0,4.617373e+08,3.821861e+08,2.143628e+08,...,0.0,7.955120e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.516793e+08
9,BEIRA,2017.0,EXECUÇÃO,3.214281e+08,2.921811e+08,2.663360e+07,2613400.0,3.819130e+08,3.293462e+08,2.143628e+08,...,0.0,5.256677e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.033410e+08


In [29]:
sheet.nrows
sheet.ncols

#find column and line 

for x in range(sheet.nrows):
    print(sheet.cell_value(x,0),sheet.cell_value(x,1),sheet.cell_value(x,2))

  
  
  
 EVOLUÇÃO DA EXECUÇÃO ORÇAMENTAL DE RECEITAS E DESPESAS DO CMC - CHIBUTO 
  
  
 DESCRICAO 2013.0
  ORÇAMENTO
1.0 RECEITAS PROPRIAS 15338210.0
 RECEITAS FISCAIS 4797780.0
 RECEITAS NAO FISCAIS 10540430.0
 RENDIMENTO DE BENS MOVEIS E IMOVEIS 0.0
2.0 RECEITAS EXTERNAS ( 2,1+2,2) 58445140.0
2.1 TRANSFERENCIAS DO ESTADO 55286140.0
 FCA 23935571.67
 FIIL 10638790.0
 FUNDOS DE ESTRADAS 7705000.0
 FUNDO CAPITAL 0.0
 OUTRAS TRANSFERENCIAS DO ESTADO 13006778.33
 FRA 0.0
2.2 FINANCIAMENTO EXTERNO - (CREDITOS+DONATIVOS+EMPRESBANCARIOS) 3159000.0
 PROMAPUTO 0.0
 OUTROS 0.0
 PCMC 3159000.0
 COOP.  ESPANHOLA 0.0
 ORCAMENTACAO PARTICIPATIVA 0.0
 P13 0.0
 PDA 0.0
 PRODEM 0.0
 UNIAO EUROPEIA 0.0
3.0 TOTAL DE RECEITAS PROPRIAS (1+2) 73783350.0
 DESPESAS 
 DESCRICAO 2013.0
  ORÇAMENTO
1.0 Deespesas Correntes 52085360.0
 Despesas com Pessoal 33577570.0
 Bens e Servicos 17314790.0
 Outras Despesas Correntes 1193000.0
2.0 Despesas de Capital 21697990.0
 Investimento Fixo 21697990.0
3.0 TOTAL DE DES